# RNN(LSTM) 함수화

In [1]:
import tensorflow as tf
import numpy as np

In [165]:
# 읽은 다음 파이썬 2와 호환되도록 디코딩합니다.
text = open('/content/drive/MyDrive/neat_mall.txt', 'rb').read().decode(encoding='utf-8')
# text = open('/content/drive/MyDrive/filter_긴팔티셔츠_text.txt', 'rb').read().decode(encoding='utf-8')

# 텍스트의 길이는 그 안에 있는 문자의 수입니다.
print ('텍스트의 길이: {}자'.format(len(text)))

텍스트의 길이: 563703자


In [166]:
# 텍스트의 처음 250자를 살펴봅니다
print(text[:250])

 데일리한 에스닉 무드를 연출해드릴 랩스커트 + 니트 민소매 투피스 세트 -  ♥   에스닉한 매력으로 예쁨 가득한 랩스커트 + 니트 민소매 투피스 세트예요 .
바캉스룩 , 리조트룩 , 휴양지룩 등 썸머 시즌에 딱 어울리는 세트 아이템 ! + 세트로 구성되어 상 , 하의를 각각 따로 매치하여 코디하셔도 좋답니다
: ) ♥  상의의 경우 니트 패브릭의 민소매 디자인과 반폴라 디테일로 완성되었어요 .
신축성이 좋아 몸에 감기며 , 편안하게 착


In [167]:
# 파일의 고유 문자수를 출력합니다.
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

고유 문자수 1076개


In [168]:
# 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [169]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '#' :   4,
  '%' :   5,
  '&' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  ':' :  16,
  ';' :  17,
  '<' :  18,
  '>' :  19,
  ...
}


In [170]:
# 텍스트에서 처음 20개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:20]), text_as_int[:20]))

' 데일리한 에스닉 무드를 연출해드릴 ' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [   2  269  741  394 1021    2  673  602  241    2  441  295  390    2
  681  868 1027  295  397    2]


In [171]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

 
데
일
리
한
 
에
스
닉
 


In [172]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

' 데일리한 에스닉 무드를 연출해드릴 랩스커트 + 니트 민소매 투피스 세트 -  ♥   에스닉한 매력으로 예쁨 가득한 랩스커트 + 니트 민소매 투피스 세트예요 .\r\n바캉스룩 , 리조트'
'룩 , 휴양지룩 등 썸머 시즌에 딱 어울리는 세트 아이템 ! + 세트로 구성되어 상 , 하의를 각각 따로 매치하여 코디하셔도 좋답니다\r\n: ) ♥  상의의 경우 니트 패브릭의 민소매'
' 디자인과 반폴라 디테일로 완성되었어요 .\r\n신축성이 좋아 몸에 감기며 , 편안하게 착용하실 수 있답니다 .\r\n스커트의 경우 , 안감 디테일이 없이 랩 디자인으로 완성되었어요 .\r\n'
'에스닉한 무드의 패턴이 페미닌한 무드를 연출해드리며 , 체형에 구애 없이 입기 좋아 꼭 ! 추천해드리고 싶어요 ~ ♥  free 사이즈로 준비되었으니 하단의 상세사이즈를 참고해주세요'
' .\r\n~ 반 사이즈까지 추천드려요 .\r\n\r\n: ) ★첫 세탁시 드라이 크리닝을 권해드리며 이후에는 단독세탁을 하셔도 무방하답니다 .\r\n 상의 -  +  하의 -  두께 : 얇음 ,'
' 비침 : 없음 신축 : 중간 ( 상의 ) , 안감 : 없음 세탁법 : 손세탁 드라이 단독세탁 울세탁 \r\n 보송한 생크림같은 촉감 화사한 파스텔리 컬러 구성의 베이직 라운드넥 부소매'
' 니트탑 -  ♥  이 중에서 고객님이 좋아하는 컬러 , 필요한 컬러가 한두개는 있을 걸요 ! ?\r\n시즌에 어울리는 화사한 컬러 구성의 보들보들 생크림 감촉 데일리 니트웨어♡ 스커트'
' , 데님진 , 슬랙스 등등 어디에나 잘 어울리기 때문에 캐쥬얼룩은 물론 오피스룩 , 로맨틱룩 등 어떤 코디에도 자유롭게 코디하기 좋은 제품이에요\r\n: ) ♥  깔끔한 룩을 연출해드'
'리며 단독 , 레이어드 어디든 매치 가능한 베이직 라운드넥 디자인이랍니다 .\r\n늘어지지 않고 복원력이 좋은 쫀쫀함에 , 표면이 정말 보송보송 부드러워서 계절 내내 손이 갈 제품이에요'
' .\r\n부담스럽게 너무 타이트하지도 않고 벙벙하게 루즈하지도 않

In [173]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [174]:
for input_example, target_example in  dataset.take(1):
  print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

입력 데이터:  ' 데일리한 에스닉 무드를 연출해드릴 랩스커트 + 니트 민소매 투피스 세트 -  ♥   에스닉한 매력으로 예쁨 가득한 랩스커트 + 니트 민소매 투피스 세트예요 .\r\n바캉스룩 , 리조'
타깃 데이터:  '데일리한 에스닉 무드를 연출해드릴 랩스커트 + 니트 민소매 투피스 세트 -  ♥   에스닉한 매력으로 예쁨 가득한 랩스커트 + 니트 민소매 투피스 세트예요 .\r\n바캉스룩 , 리조트'


In [175]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 2 (' ')
  예상 출력: 269 ('데')
   1단계
  입력: 269 ('데')
  예상 출력: 741 ('일')
   2단계
  입력: 741 ('일')
  예상 출력: 394 ('리')
   3단계
  입력: 394 ('리')
  예상 출력: 1021 ('한')
   4단계
  입력: 1021 ('한')
  예상 출력: 2 (' ')


In [176]:
# 배치 크기
BATCH_SIZE = 64

# 데이터셋을 섞을 버퍼 크기
# (TF 데이터는 무한한 시퀀스와 함께 작동이 가능하도록 설계되었으며,
# 따라서 전체 시퀀스를 메모리에 섞지 않습니다. 대신에,
# 요소를 섞는 버퍼를 유지합니다).
BUFFER_SIZE = 5000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [177]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [178]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [179]:
model = build_model(vocab_size = len(vocab),
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

In [180]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

(64, 100, 1076) # (배치 크기, 시퀀스 길이, 어휘 사전 크기)


In [181]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (64, None, 256)           275456    
_________________________________________________________________
lstm_14 (LSTM)               (64, None, 1024)          5246976   
_________________________________________________________________
dense_14 (Dense)             (64, None, 1076)          1102900   
Total params: 6,625,332
Trainable params: 6,625,332
Non-trainable params: 0
_________________________________________________________________


In [182]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [183]:
sampled_indices

array([ 628, 1040,  369,  453,  335,  308,  515,  763, 1030,  497,  337,
        515,  211,  492,  616,  390,  141,  858,  422,  237,  599,  725,
        981,  809,  804,  539,   97,  836, 1049,  836,  335,  328,  524,
        293,  695,  632,  427,  522,  265,  866,  924,  924,  919,  342,
        470,  695,  862,  917,   23,  513,  111,  599,  949,  963,  324,
        800,  830,  498,  434,  949,  353,  548,  239,  743,  524,  981,
        665,  143,  701,  472,  984,  602,  839,  433,  168,  543,  590,
         25,  342,  710,  271,  139,  822,  194,   79,  752,  140,  206,
        631,  966,  369,  813,  884,  161,  584,  233, 1014,   21,  761,
        208])

In [184]:
print("입력: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices ])))

입력: 
 ' 있으며 , 오픈하거나 내츄럴하게 걸쳤을때에 더 예쁜 아이템이에요\r\n♡ s / s 시즌내내 입고 다니게 될 데일리 베이직 아이템 , 지금 만나보세요 ~ ♥  free 사이즈로 준비'

예측된 다음 문자: 
 '쏙혔렷미랄따빅적했복랍빅널변십를글촘먼늘쉬윈패짙질쁜겉참환참랄뜸뺀뒷올쓰멜빵덥축키키큐래밤올촬퀘]블고쉬토특뚫증찜본몫토럴삼늬입뺀패언금왔방팽스찾목꼬삐숏a래우델극쫀낙갓잖근냉쑥틀렷짧칩껄솟뉘픽[쟁냥'


In [185]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실:          ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 1076)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:           6.981339


In [186]:
model.compile(optimizer='adam', loss=loss)

In [187]:
# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [188]:
history = model.fit(dataset, epochs=30, callbacks=[checkpoint_callback])

Epoch 1/30
87/87 [==============================] - 8s 75ms/step - loss: 5.0397
Epoch 2/30
87/87 [==============================] - 7s 75ms/step - loss: 3.7862
Epoch 3/30
87/87 [==============================] - 7s 76ms/step - loss: 2.6211
Epoch 4/30
87/87 [==============================] - 7s 77ms/step - loss: 2.0838
Epoch 5/30
87/87 [==============================] - 7s 77ms/step - loss: 1.8487
Epoch 6/30
87/87 [==============================] - 7s 77ms/step - loss: 1.7092
Epoch 7/30
87/87 [==============================] - 7s 78ms/step - loss: 1.5767
Epoch 8/30
87/87 [==============================] - 7s 78ms/step - loss: 1.5094
Epoch 9/30
87/87 [==============================] - 7s 78ms/step - loss: 1.4489
Epoch 10/30
87/87 [==============================] - 7s 79ms/step - loss: 1.3998
Epoch 11/30
87/87 [==============================] - 7s 79ms/step - loss: 1.3296
Epoch 12/30
87/87 [==============================] - 7s 79ms/step - loss: 1.3025
Epoch 13/30
87/87 [==================

In [189]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [190]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [191]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (1, None, 256)            275456    
_________________________________________________________________
lstm_15 (LSTM)               (1, None, 1024)           5246976   
_________________________________________________________________
dense_15 (Dense)             (1, None, 1076)           1102900   
Total params: 6,625,332
Trainable params: 6,625,332
Non-trainable params: 0
_________________________________________________________________


In [194]:
def generate_text(model, start_string):
  # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

  # 생성할 문자의 수
  num_generate = 1000

  # 시작 문자열을 숫자로 변환(벡터화)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 빈 문자열
  text_generated = []

  # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
  # 온도가 높으면 더 의외의 텍스트가 됩니다.
  # 최적의 세팅을 찾기 위한 실험
  temperature = 0.2

  # 여기에서 배치 크기 == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 배치 차원 제거
      predictions = tf.squeeze(predictions, 0)

      # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 예측된 단어를 다음 입력으로 모델에 전달
      # 이전 은닉 상태와 함께
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [195]:
print(generate_text(model, start_string=u"사이즈"))

사이즈표를 꼭 확인 후 초이스해주세요 .
* 좋은 소재 , 오랫 동안 예쁘게 유지되도록 세탁기 , 세탁건조기 사용은 무조건 피해주시고 , 물세탁을 하실 경우에는 따뜻하지 않은 미온수에 울세제로 간단한 단독 손세탁을 권장드려요 .
■   +  + 
 ( s ) / shoriating % 두께 : 중간 , 비침 : 없음 신축 : 높음 , 안감 : 없음 세탁법 : 손세탁 드라이 단독세탁 울세탁 세탁기금지 세탁건조기 사용불가 
 고급스러운 무드를 선사할 니트 가디건 소개해드릴게요 : ) 부드러운 터치감으로 피부가 예민한 고객님들도 부담없이 착용 가능하답니다 .
♥  free 사이즈로 준비되었으니 하단의 상세사이즈를 참고해주세요 .
~ 반 사이즈까지 추천드려요 .
* 사이즈 , 핏감은 체형에 따라 개인차가 있다는 점 감안해주세요
: )   두께 : 중간 , 비침 : 중간 신축 : 높음 , 안감 : 없음 세탁법 : 손세탁 드라이 단독세탁 울세탁 세탁기금지 세탁건조기 사용불가 
 고급스러운 진주 버튼 포인트의 루즈핏 니트 -  ♥  고객님들의 사랑을 받고 있는 ts , mint 니트 가디건 ! 이번에는 단독으로 입어도 좋은 아이템이에요 .
♥  free 사이즈로 준비되었으니 하단의 상세사이즈를 참고해주세요 .
~ 반 사이즈까지 추천드려요 .
* 사이즈 , 핏감은 체형에 따라 개인차가 있다는 점 감안해주세요
: )   두께 : 중간 , 비침 : 없음 신축 : 높음 , 안감 : 없음 세탁법 : 손세탁 드라이 단독세탁 울세탁 세탁기금지 세탁건조기 사용불가 
 여리한 컬러 배색♥  ♥  깔끔한 컬러감의 배색 디자인의 니트 가디건 소개해드려요 .
가지 컬러로 준비했으니 굿초이스하세요 - ♪ ♥  free 사이즈로 준비되었으니 하단의 상세사이즈를 참고해주세요 .
~ 반 사이즈까지 추천드려요 .
* 사이즈 , 핏감은 체형에 따라 개인차가 있다는 점 감안해주세요
: )   +  두께 : 중간 , 비침 : 없음 신축 : 높음 , 안감 : 없음 세탁법 : 손세탁 
